# <center>Organizaci&oacute;n de Datos</center>
## <center>Trabajo Práctico 2 </center>

## <center>Grupo 21</center>

#### Integrantes: Francisco Juárez, Lucas Aldazabal, Valentin Schneider, Gonzalo Rizzo Ehrenbock

# <center> Procesamiento del Lenguaje Natural <center/> 

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sns.set_theme(style="whitegrid")

In [32]:
#Import dataframe
ds_propiedades = pd.read_csv('./ds_propiedades_total_clasificado.csv')
ds_descripciones = pd.read_csv('./properati_argentina_2021_decrip.csv')
ds_propiedades.head()

,id,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title,pxm2,tipo_precio
0,F5IvmD1uzzdIR7b96ozt4g==,-34.558855,-58.452458,Nuñez,Departamento,4.0,3.0,160.0,146.0,460000.0,Venta: Hermoso piso de categoría en excelente ...,2875.000000,alto
1,IiT/P7WfYDcqbhu7tf5V4Q==,-34.582403,-58.443841,Colegiales,Departamento,2.0,1.0,50.0,46.0,148000.0,Departamento - Colegiales,2960.000000,alto
2,6XXcElx/4OwCcH4NI06Oiw==,-34.605825,-58.422214,Almagro,Departamento,3.0,2.0,47.0,47.0,89000.0,Departamento en venta tres ambientes- Almagro,1893.617021,medio
3,8R7hSzqWFk9rX4wBXxfORA==,-34.561971,-58.455670,Belgrano,Departamento,2.0,1.0,52.0,48.0,127000.0,Departamento - Belgrano,2442.307692,medio
4,UAKYqQx9s0MfIdLzrJv8Ig==,-34.606738,-58.389426,Tribunales,Departamento,3.0,2.0,76.0,75.0,129900.0,Lindísimo departamento antiguo en excelente es...,1709.210526,bajo


In [33]:
#filter ds_descripciones by id in ds_propiedades
ds_descripciones = ds_descripciones[ds_descripciones['id'].isin(ds_propiedades['id'])]
ds_descripciones.head()

,id,property_description
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...
2,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...
3,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,..."
10,+9uGr8tPEUkKnpjOef+gUw==,SE VENDE EXCELENTE PH de 2 CÓMODOS AMBIENTES M...
11,uH5H+RfQM87Qtrb13+T4hA==,"Venta PH 4 ambientes Paternal<br><br>Cálido, ..."


## Exportamos el dataframe a un archivo de texto para poder pasarlo por ECmes

In [34]:
ds_descripciones_sin_id = ds_descripciones.drop(['id'], axis=1)

ds_descripciones_sin_id.to_csv('ds_descripciones.txt', index=False)

## Limpiamos el archivo de descripciones para que lo pueda leer Ecmes correctamente.

In [35]:
dic_replace = {
    '(': ',',
    ')': ',',
    '<br>': ' ',
    '<br/>': ' ',
    '/': ' ',
    '°': ' ',
    'º': ' ',
    ';': ' ',
    '•': ' ',
    ':': ' ',
    '#': ' ',
    '•': ' ',
    '-': ' ',
    '_': ' ',
    '"': ' ',
    'á': 'a',
    'é': 'e',
    'í': 'i',
    'ó': 'o',
    'ú': 'u',
    'Á': 'A',
    'É': 'E',
    'Í': 'I',
    'Ó': 'O',
    'Ú': 'U',
    'ñ': 'ni',
    'Ñ': 'NI',
    '!': ' ',
    'm²': 'm2',
    'M²': 'M2',
    '&': 'y',
}

In [36]:
#for every line in ds_descripciones.txt replace the '(' and ')' with ',' and save it in ds_descripciones.txt line by line 
with open('ds_descripciones.txt', 'r', encoding = 'utf8') as f_in, open('ds_descripciones_out.txt', 'w', encoding = 'utf8') as f_out:
    for line in f_in:
        for src, target in dic_replace.items():
            line = line.replace(src, target)
        f_out.writelines(line)


    



##### por ahora no usamos esto, pero capaz lo necesitamos en algun momento

In [37]:
# count how many times every word appears in ds_descripciones['property_description']
# and save it in a dictionary


word_count = {}
palabras_ignoradas = ['para', 'baño', 'cocina', 'comedor','casa','cuenta','esta','living']
ds_descripciones.reset_index(drop=True, inplace=True)

for i in range(len(ds_descripciones['property_description'])):
    if type(ds_descripciones['property_description'][i]) == str:
        for word in ds_descripciones['property_description'][i].split():
            if len(word) < 4 or word in palabras_ignoradas:
                continue
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1

# save dictionary in a dataframe

word_count_aux = word_count.copy()

for key, value in word_count_aux.items():
    if value <= 300:
        del word_count[key]

df_word_count = pd.DataFrame.from_dict(word_count, orient='index', columns=['count'])
df_word_count.sort_values(by=['count'], ascending=False, inplace=True)
df_word_count



# barplot of the 20 most common words
plt.figure(figsize=(20,10))
sns.barplot(x=df_word_count.index[:100], y=df_word_count['count'][:100])
plt.xticks(rotation=90)
plt.show()


KeyboardInterrupt: 

In [ ]:
##split train y test
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(ds_descripciones_sin_id, ds_propiedades['tipo_precio'], test_size=0.20, random_state=1)

## 1b) Entranamiento XGBoost
### Hiperparametros TP1

In [ ]:
import xgboost as xgb

#Creo un modelo de XGBoost para reductir el error cuadraditico medio
xgb.XGBRegressor().get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [ ]:
xgb_model = xgb.XGBRegressor(objective = 'reg:squarederror',
 base_score= None,
 booster= None,
 callbacks= None,
 colsample_bylevel= None,
 colsample_bynode= None,
 colsample_bytree= None,
 early_stopping_rounds= None,
 enable_categorical= False,
 eval_metric= None,
 feature_types= None,
 gamma= None,
 gpu_id= None,
 grow_policy= None,
 importance_type= None,
 interaction_constraints= None,
 learning_rate= None,
 max_bin= None,
 max_cat_threshold= None,
 max_cat_to_onehot= None,
 max_delta_step= None,
 max_depth= None,
 max_leaves= None,
 min_child_weight= None,
 missing= "nan",
 monotone_constraints= None,
 n_estimators= 100,
 n_jobs= None,
 num_parallel_tree= None,
 predictor= None,
 random_state= None,
 reg_alpha= None,
 reg_lambda= None,
 sampling_method= None,
 scale_pos_weight= None,
 subsample= None,
 tree_method= None,
 validate_parameters= None,
 verbosity= None)

In [ ]:
xgb_model.fit(x_train, y_train)

y_pred = xgb_model.predict(x_test)

In [ ]:
mse = metrics.mean_squared_error(
        y_true  = y_test,
        y_pred  = y_pred,
        squared = True
         )

print(f"El error (mse) de test es: {mse}")



rmse = metrics.mean_squared_error(
        y_true  = y_test,
        y_pred  = y_pred,
        squared = False
         )

print(f"El error (rmse) de test es: {rmse}")  

In [ ]:
#Coef. de Determinación R2
score_xgb_train = xgb_model.score(x_train, y_train)
score_xgb_test = xgb_model.score(x_test, y_test)
print(f"El coeficiente de Determinación R2 en train es: {score_xgb_train}")
print(f"El coeficiente de Determinación R2 en test es: {score_xgb_test}")

In [ ]:
# n=10

# #xgb param grid
# xgb_param_grid = {
#     'n_estimators': [50, 100],
#     'learning_rate': [0.1, 0.3],
#     'max_depth': [3, 7],
#     'gamma': [0.3, 0.4],
# }

# folds=5
# kfoldcv = StratifiedKFold(n_splits=folds, random_state=10,shuffle=True)

# scorer_fn = make_scorer(f1_score, average='micro')

# random_search_cv = RandomizedSearchCV(estimator=xgb_cv,
#                               param_distributions = xgb_param_grid,
#                               scoring=scorer_fn,
#                               cv=kfoldcv,
#                               n_iter=n,
#                               random_state=10) 

# random_fit = random_search_cv.fit(ds_propiedades_train_x, ds_propiedades_train_y)

In [ ]:
# print(random_fit.best_params_)